Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

In [1]:
# today is about reps
# We already know how to do the stuff

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

*Note:* Each time we call the pipeline during grid search, each component is fit again. The vectorizer (tf-idf) is transforming our entire vocabulary during each cross-validation fold. That transformation adds significant run time to our grid search. There *might* be interactions between the vectorizer and our classifier, so we estimate their performance together in the code below. However, if your goal is to reduce run time. Train your vectorizer separately (ie out of the grid-searched pipeline). 

In [4]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [7]:
#print out data sample
print(data['data'][1])

Subject: Re: There must be a creator! (Maybe)
From: halat@pooh.bears (Jim Halat)
Reply-To: halat@pooh.bears (Jim Halat)
Lines: 24

In article <16BA1E927.DRPORTER@SUVM.SYR.EDU>, DRPORTER@SUVM.SYR.EDU (Brad Porter) writes:
>
>   Science is wonderful at answering most of our questions.  I'm not the type
>to question scientific findings very often, but...  Personally, I find the
>theory of evolution to be unfathomable.  Could humans, a highly evolved,
>complex organism that thinks, learns, and develops truly be an organism
>that resulted from random genetic mutations and natural selection?

[...stuff deleted...]

Computers are an excellent example...of evolution without "a" creator.
We did not "create" computers.  We did not create the sand that goes
into the silicon that goes into the integrated circuits that go into
processor board.  We took these things and put them together in an
interesting way. Just like plants "create" oxygen using light through 
photosynthesis.  It's a much bigger 

In [ ]:
# Create Pipeline Components

# ngram is one you always want to specify and tune

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [ ]:
# Define the Pipeline

# there's also make_pipe, JC likes this
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

In [ ]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

## Challenge

You're trying to achieve 75% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [9]:
# latent semantic indexing = tfidf + (thing. truncatedSVD i think)

# we're gonna make lsi a pipeline and stick it into the other one later
# so we can tune everything at once

# turning off min_df helps us pick up class 2's in the assignment

In [10]:
# Import

from sklearn.decomposition import TruncatedSVD

#if you want to tune anything in here, do n_components
svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [13]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators':[5,10,20],
    'lsi__vect__ngram_range':[(1,1),(1,2),(1,3)]
}

In [14]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

NameError: name 'vect' is not defined

In [ ]:
print(pipe)

In [ ]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

In [ ]:
grid_search.best_score_

In [ ]:
# You want ~10k observations per class to use a neural network.
# So ~20,000 observations in a binary classification problem

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [15]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [16]:
doc = nlp("Two bananas in pyjamas")

In [17]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [18]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [19]:
X = get_word_vectors(train['description'])

len(X) == len(data.data)

NameError: name 'train' is not defined

In [ ]:
X_test = get_word_vectors(test['description'])

In [ ]:
rfc.fit(X, train['ratingCategory'])

In [ ]:
rfc.score(X, train['ratingCategory'])

In [ ]:
rfc.predict(X_test)

In [ ]:
test['ratingCategory'] = rfc.predict(X_test)

In [ ]:
test[['id', 'ratingCategory']].to_csv('testSolutionSubmission.csv', header=True, index=False)

## Follow Along

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle competition
* Find another text classification task to work on